Nota generada a partir de [liga](https://www.dropbox.com/s/yjijtfuky3s5dfz/2.5.Compute_Unified_Device_Architecture.pdf?dl=0)

**Notas para contenedor de docker:**

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

```
docker run --gpus all --rm -v $(pwd):/datos --name jupyterlab_nvidia_cuda_c_container -p 8888:8888 -d palmoreck/jupyterlab_nvidia_cuda_c:1.1.0_10.2
```

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

```
docker stop jupyterlab_nvidia_cuda_c_container
```

Documentación de la imagen de docker `palmoreck/jupyterlab_nvidia_cuda_c:1.1.0_10.2` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/nvidia/cuda_c).

---

**Nota: si se desean ejecutar los ejemplos que se presentan a continuación, es necesario tener una tarjeta gráfica NVIDIA.**

# CUDA C y generalidades de CUDA y GPU

## CUDA C

Consiste en extensiones al lenguaje C y en una *runtime library*. Ver [2.3.CUDA](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/II.computo_paralelo/2.3.CUDA.ipynb) para más información.

### Kernel

* En CUDA C se define una función que se ejecuta en el device y que se le nombra **kernel**. El *kernel* inicia con la sintaxis:

```
__global__ void mifun(int param){
...
}

```

y siempre es tipo `void` (no hay `return`).

* El llamado al *kernel* se realiza desde el host y con una sintaxis en la que se define el número de threads y bloques que serán utilizados para la ejecución del kernel. La sintaxis que se utiliza es con `<<< >>>` y en la primera entrada se coloca el número de bloques y en la segunda entrada el número de *threads*:


```
__global__ void mifun(int param){
...
}

int main(){
    int par;
    mifun<<<N,5>>> (par); //N bloques de 5 threads
}

```

## Ejemplos

### 1) Programa de hello world

In [1]:
%%file hello_world.cu
#include<stdio.h>
__global__ void func(void){
}
int main(void){
    func<<<1,1>>>(); //1 bloque de 1 thread
    printf("Hello world!\n");
return 0;
}

Writing hello_world.cu


Compilación:

In [2]:
%%bash
nvcc --compiler-options -Wall hello_world.cu -o hello_world.out

Ejecución:

In [3]:
%%bash
./hello_world.out

Hello world!


**Comentarios:**

* La función `main` se ejecuta en la CPU.

* La función `func` es un *kernel* y es ejecutada por *threads* en el *device* (GPU), también llamados **CUDA threads**. Obsérvese que la función `func` inicia con `__global__` para diferenciarla de `main`. En este caso el *thread* que fue lanzado no realiza ninguna acción pues el cuerpo del kernel está vacío.

* El *kernel* sólo puede tener un `return` tipo *void*: `__global__ void func` por lo que el *kernel* debe regresar sus resultados a través de sus argumentos.
 

### 2) Programa de hello world 2

In [4]:
%%file hello_world_2.cu
#include<stdio.h>
__global__ void func(void){
    printf("Hello world del bloque %d del thread %d!\n", blockIdx.x, threadIdx.x);
}
int main(void){
    func<<<2,3>>>(); //2 bloques de 3 threads cada uno
    cudaDeviceSynchronize();
    printf("Hola del cpu thread\n");
    return 0;
}


Writing hello_world_2.cu


In [5]:
%%bash 
nvcc --compiler-options -Wall hello_world_2.cu -o hello_world_2.out

In [6]:
%%bash
./hello_world_2.out

Hello world del bloque 0 del thread 0!
Hello world del bloque 0 del thread 1!
Hello world del bloque 0 del thread 2!
Hello world del bloque 1 del thread 0!
Hello world del bloque 1 del thread 1!
Hello world del bloque 1 del thread 2!
Hola del cpu thread


**Comentarios:**

* La extensión del archivo debe ser `.cu` aunque esto puede modificarse al compilar con `nvcc`: 

`$nvcc -x cu hello_world.c -o hello_world.out`

* El llamado a la ejecución del kernel se realizó en el *host* y se lanzaron $2$ bloques (primera posición), cada uno con $3$ *threads*.

* Se utiliza la función [cudaDeviceSynchronize](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__DEVICE.html#group__CUDART__DEVICE_1g10e20b05a95f638a4071a655503df25d) para que el *cpu-thread* espere la finalización de la ejecución del kernel.

* Los *CUDA threads* son divididos en bloques, **CUDA blocks** y todos los bloques se encuentran en un **grid**. En el lanzamiento del *kernel*  se debe especificar al hardware cuántos bloques tendrá nuestro *grid* y cuántos *threads* estarán en cada bloque. Las variables `blockIdx` y `threadIdx` hacen referencia a los **id**'s que tienen los bloques y los threads. El *id* del bloque dentro del *grid* y el *id* del thread dentro del bloque. La parte `.x` de `blockIdx.x` y `threadIdx.x` refiere a la **primera coordenada** del bloque en el *grid* y del *thread* en en el bloque. 

* La elección del número de bloques en un grid o el número de *threads* en un bloque no corresponde a alguna disposición del hardware, esto es, si se lanza un kernel con `<<< 1, 3 >>>` no implica que la GPU tenga en su hardware un bloque o 3 *threads*.

* En una GPU podemos definir el *grid* de bloques y el bloque de *threads* utilizando el tipo de dato `dim3` el cual también es parte de CUDA C:

In [7]:
%%file hello_world_3.cu
#include<stdio.h>
__global__ void func(void){
    printf("Hello world del bloque %d del thread %d!\n", blockIdx.x, threadIdx.x);
}
int main(void){
    dim3 dimGrid(2,1,1); //2 bloques en el grid
    dim3 dimBlock(3,1,1); //3 threads por bloque
    func<<<dimGrid,dimBlock>>>(); 
    cudaDeviceSynchronize();
    printf("Hola del cpu thread\n");
    return 0;
}

Writing hello_world_3.cu


In [8]:
%%bash 
nvcc --compiler-options -Wall hello_world_3.cu -o hello_world_3.out

In [9]:
%%bash
./hello_world_3.out

Hello world del bloque 1 del thread 0!
Hello world del bloque 1 del thread 1!
Hello world del bloque 1 del thread 2!
Hello world del bloque 0 del thread 0!
Hello world del bloque 0 del thread 1!
Hello world del bloque 0 del thread 2!
Hola del cpu thread


**Obs:** obsérvese que puede definirse un grid de tres dimensiones y también un bloque de tres dimensiones.

In [174]:
%%file thread_idxs.cu
#include<stdio.h>
__global__ void func(void){
    if(threadIdx.x==0 && threadIdx.y==0 && threadIdx.z==0){
        printf("blockIdx.x:%d\n",blockIdx.x);
    }
    printf("thread idx.x:%d\n",threadIdx.x);
    printf("thread idx.y:%d\n",threadIdx.y);
    printf("thread idx.z:%d\n",threadIdx.z);
}
int main(void){
    dim3 dimGrid(1,1,1); //1 bloque en el grid
    dim3 dimBlock(1,3,1); //3 threads por bloque
    func<<<dimGrid,dimBlock>>>(); 
    cudaDeviceSynchronize();
    return 0;
}

Overwriting thread_idxs.cu


In [175]:
%%bash 
nvcc --compiler-options -Wall thread_idxs.cu -o thread_idxs.out

In [176]:
%%bash
./thread_idxs.out

blockIdx.x:0
thread idx.x:0
thread idx.x:0
thread idx.x:0
thread idx.y:0
thread idx.y:1
thread idx.y:2
thread idx.z:0
thread idx.z:0
thread idx.z:0


In [148]:
%%file block_idxs.cu
#include<stdio.h>
__global__ void func(void){
    printf("blockIdx.x:%d\n",blockIdx.x);
    printf("blockIdx.y:%d\n",blockIdx.y);
    printf("blockIdx.z:%d\n",blockIdx.z);

}
int main(void){
    dim3 dimGrid(1,2,2); //4 bloques en el grid
    dim3 dimBlock(1,1,1); //1 thread por bloque
    func<<<dimGrid,dimBlock>>>(); 
    cudaDeviceSynchronize();
    return 0;
}

Overwriting block_idxs.cu


In [149]:
%%bash 
nvcc --compiler-options -Wall block_idxs.cu -o block_idxs.out

In [150]:
%%bash
./block_idxs.out

blockIdx.x:0
blockIdx.x:0
blockIdx.x:0
blockIdx.x:0
blockIdx.y:0
blockIdx.y:1
blockIdx.y:0
blockIdx.y:1
blockIdx.z:0
blockIdx.z:1
blockIdx.z:1
blockIdx.z:0


In [183]:
%%file block_dims.cu
#include<stdio.h>
__global__ void func(void){
    if(threadIdx.x==0 && threadIdx.y==0 && threadIdx.z==0 && blockIdx.z==1){
    printf("blockDim.x:%d\n",blockDim.x);
    printf("blockDim.y:%d\n",blockDim.y);
    printf("blockDim.z:%d\n",blockDim.z);
    }

}
int main(void){
    dim3 dimGrid(2,2,2); //8 bloques en el grid
    dim3 dimBlock(3,1,2); //6 threads por bloque
    func<<<dimGrid,dimBlock>>>(); 
    cudaDeviceSynchronize();
    return 0;
}

Overwriting block_dims.cu


In [184]:
%%bash 
nvcc --compiler-options -Wall block_dims.cu -o block_dims.out

In [185]:
%%bash
./block_dims.out

blockDim.x:3
blockDim.x:3
blockDim.x:3
blockDim.x:3
blockDim.y:1
blockDim.y:1
blockDim.y:1
blockDim.y:1
blockDim.z:2
blockDim.z:2
blockDim.z:2
blockDim.z:2


### 3) Programa de suma vectorial

**N bloques de 1 thread**

In [10]:
%%file suma_vectorial.cu
#include<stdio.h>
#define N 10
__global__ void suma_vect(int *a, int *b, int *c){
    int block_id_x = blockIdx.x;
    if(block_id_x<N) //aquí se asume que el valor de N 
                     //es menor al número máximo de bloques que se pueden lanzar
                     //si fuese mayor, hay que hacer un ajuste
        c[block_id_x] = a[block_id_x]+b[block_id_x];
}
int main(void){
    int a[N], b[N],c[N];
    int *device_a, *device_b, *device_c;
    int i;
    dim3 dimGrid(N,1,1); //N bloques en el grid
    dim3 dimBlock(1,1,1); //1 threads por bloque 
    //alojando en device
    cudaMalloc((void **)&device_a, sizeof(int)*N); 
    cudaMalloc((void **)&device_b, sizeof(int)*N);
    cudaMalloc((void **)&device_c, sizeof(int)*N);
    //llenando los arreglos con datos dummy:
    for(i=0;i<N;i++){
        a[i]=i;
        b[i]=i*i;
    }
    //copiamos arreglos a, b a la GPU
    cudaMemcpy(device_a,a,N*sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(device_b,b,N*sizeof(int), cudaMemcpyHostToDevice);
    //mandamos a llamar a suma_vect:
    suma_vect<<<dimGrid,dimBlock>>>(device_a,device_b,device_c); //N bloques de 1 thread
    cudaDeviceSynchronize();
    //copia del resultado al arreglo c:
    cudaMemcpy(c,device_c,N*sizeof(int),cudaMemcpyDeviceToHost);
    for(i=0;i<N;i++)
        printf("%d+%d = %d\n",a[i],b[i],c[i]);
    cudaFree(device_a);
    cudaFree(device_b);
    cudaFree(device_c);
    return 0;
}

Writing suma_vectorial.cu


In [11]:
%%bash
nvcc --compiler-options -Wall suma_vectorial.cu -o suma_vectorial.out

In [12]:
%%bash
./suma_vectorial.out

0+0 = 0
1+1 = 2
2+4 = 6
3+9 = 12
4+16 = 20
5+25 = 30
6+36 = 42
7+49 = 56
8+64 = 72
9+81 = 90


**Comentarios:**

* Obsérvese que se están utilizando apuntadores en la línea:

```
    int *device_a, *device_b, *device_c;
```

pero estos apuntadores no apuntan a una dirección de memoria en el *device* pues aunque NVIDIA añadió el *feature* de [Unified Memory](https://devblogs.nvidia.com/unified-memory-cuda-beginners/) (un espacio de memoria accesible para la CPU y la GPU) aquí no se está usando tal *feature*. Más bien se están utilizando los apuntadores anteriores para apuntar a un [struct](https://en.wikipedia.org/wiki/Struct_(C_programming_language)) de C en el que uno de sus tipos de datos es una dirección de memoria en la GPU.

* Para alojar memoria en la GPU se utiliza el llamado a [cudaMalloc](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__MEMORY.html#group__CUDART__MEMORY_1g37d37965bfb4803b6d4e59ff26856356) y para transferir datos del *host* al *device* o viceversa se llama a lafunción [cudaMemcpy](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__MEMORY.html#group__CUDART__MEMORY_1gc263dbe6574220cc776b45438fc351e8) con respectivos parámetros como `cudaMemcpyHostToDevice` o `cudaMemcpyDeviceToHost`. Obsérvese el uso de `(void **)` por la definición de la función `cudaMalloc`.

* Para desalojar memoria en la GPU se utiliza el llamado a [cudaFree](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__MEMORY.html#group__CUDART__MEMORY_1ga042655cbbf3408f01061652a075e094)

Al instalar el *CUDA toolkit* o con el contenedor de docker (detallado al inicio de la nota) se cuenta con la línea de comando [nvprof](https://docs.nvidia.com/cuda/profiler-users-guide/index.html) para perfilamiento (aunque en la documentación se menciona que será reemplazada tal línea de comando por [NVIDIA Nsight Compute](https://developer.nvidia.com/nsight-compute) y [NVIDIA Nsight Systems](https://developer.nvidia.com/nsight-systems))

In [13]:
%%bash 

nvprof --normalized-time-unit s ./suma_vectorial.out

0+0 = 0
1+1 = 2
2+4 = 6
3+9 = 12
4+16 = 20
5+25 = 30
6+36 = 42
7+49 = 56
8+64 = 72
9+81 = 90


==216== NVPROF is profiling process 216, command: ./suma_vectorial.out
==216== Profiling application: ./suma_vectorial.out
==216== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
                        %         s                   s         s         s
 GPU activities:    39.91  2.78e-06         1  2.78e-06  2.78e-06  2.78e-06  suma_vect(int*, int*, int*)
                    35.32  2.46e-06         2  1.23e-06  9.28e-07  1.54e-06  [CUDA memcpy HtoD]
                    24.77  1.73e-06         1  1.73e-06  1.73e-06  1.73e-06  [CUDA memcpy DtoH]
      API calls:    99.37  0.289724         3  0.096575  6.04e-06  0.289709  cudaMalloc
                     0.19  5.64e-04       194  2.91e-06  1.66e-07  1.16e-04  cuDeviceGetAttribute
                     0.16  4.71e-04         2  2.35e-04  2.17e-04  2.54e-04  cuDeviceTotalMem
                     0.11  3.17e-04         1  3.17e-04  3.17e-04  3.17e-04  cudaLaunchKernel
                     0

Unidades en las que se reporta, s: second, ms: millisecond, us: microsecond, ns: nanosecond

**1 bloque de N threads**

Y en lugar de lanzar $N$ bloques de $1$ thread se puede lanzar $1$ bloque con $N$ threads:

In [14]:
%%file suma_vectorial_2.cu
#include<stdio.h>
#define N 10
__global__ void suma_vect(int *a, int *b, int *c){
    int thread_id_x = threadIdx.x;
    if(thread_id_x<N) //aquí se asume que el valor de N 
                     //es menor al número máximo de threads que se pueden lanzar
                    //si fuese mayor, hay que hacer un ajuste
        c[thread_id_x] = a[thread_id_x]+b[thread_id_x];
}
int main(void){
    int *device_a, *device_b, *device_c;
    int i;
    dim3 dimGrid(1,1,1); //1 bloques en el grid
    dim3 dimBlock(N,1,1); //N threads por bloque 
    //alojando en device con Unified Memory
    cudaMallocManaged(&device_a, sizeof(int)*N);
    cudaMallocManaged(&device_b, sizeof(int)*N);
    cudaMallocManaged(&device_c, sizeof(int)*N);
    //llenando los arreglos:
    for(i=0;i<N;i++){
        device_a[i]=i;
        device_b[i]=i*i;
    }
    suma_vect<<<dimGrid,dimBlock>>>(device_a,device_b,device_c); //1 bloque con N threads
    cudaDeviceSynchronize();
    for(i=0;i<N;i++)
        printf("%d+%d = %d\n",device_a[i],device_b[i],device_c[i]);
    cudaFree(device_a);
    cudaFree(device_b);
    cudaFree(device_c);
    return 0;
}

Writing suma_vectorial_2.cu


In [15]:
%%bash
nvcc --compiler-options -Wall suma_vectorial_2.cu -o suma_vectorial_2.out

In [16]:
%%bash
./suma_vectorial_2.out

0+0 = 0
1+1 = 2
2+4 = 6
3+9 = 12
4+16 = 20
5+25 = 30
6+36 = 42
7+49 = 56
8+64 = 72
9+81 = 90


In [17]:
%%bash 

nvprof --normalized-time-unit s ./suma_vectorial_2.out

0+0 = 0
1+1 = 2
2+4 = 6
3+9 = 12
4+16 = 20
5+25 = 30
6+36 = 42
7+49 = 56
8+64 = 72
9+81 = 90


==266== NVPROF is profiling process 266, command: ./suma_vectorial_2.out
==266== Profiling application: ./suma_vectorial_2.out
==266== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
                        %         s                   s         s         s
 GPU activities:   100.00  3.26e-06         1  3.26e-06  3.26e-06  3.26e-06  suma_vect(int*, int*, int*)
      API calls:    98.79  0.163188         3  0.054396  1.13e-05  0.163138  cudaMallocManaged
                     0.43  7.13e-04       194  3.68e-06  1.63e-07  1.28e-04  cuDeviceGetAttribute
                     0.30  4.98e-04         1  4.98e-04  4.98e-04  4.98e-04  cudaLaunchKernel
                     0.29  4.72e-04         2  2.36e-04  2.17e-04  2.55e-04  cuDeviceTotalMem
                     0.09  1.44e-04         3  4.81e-05  1.94e-05  8.86e-05  cudaFree
                     0.08  1.30e-04         2  6.50e-05  2.30e-05  1.07e-04  cuDeviceGetName
                     0.0

**Obs:** obsérvese que el programa anterior utiliza la *Unified Memory* con la función [cudaMallocManaged](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__HIGHLEVEL.html#group__CUDART__HIGHLEVEL_1gcf6b9b1019e73c5bc2b39b39fe90816e).

El siguiente programa basado en [liga](https://devblogs.nvidia.com/how-query-device-properties-and-handle-errors-cuda-cc/) y [cudaDeviceProp Struct Reference](https://docs.nvidia.com/cuda/cuda-runtime-api/structcudaDeviceProp.html)

https://stackoverflow.com/questions/22520209/programmatically-retrieve-maximum-number-of-blocks-per-multiprocessor

In [18]:
%%file device_properties.cu

#include<stdio.h>

int main(void){
    cudaDeviceProp properties;
    int count;
    int i;
    cudaGetDeviceCount(&count);
    for(i=0;i<count;i++){
        printf("----------------------\n");
        cudaGetDeviceProperties(&properties, i);
        printf("----device %d ----\n",i); 
        printf("Device Name: %s\n", properties.name);
        printf("Compute capability: %d.%d\n", properties.major, properties.minor);
        printf("Clock rate: %d\n", properties.clockRate);
        printf("Unified memory: %d\n", properties.unifiedAddressing);
        printf(" ---Memory Information for device %d (results on bytes)---\n", i);
        printf("Total global mem: %ld\n", properties.totalGlobalMem); 
        printf("Total constant Mem: %ld\n", properties.totalConstMem);
        printf("Shared memory per thread block: %ld\n", properties.sharedMemPerBlock);
        printf("Shared memory per SM: %ld\n",properties.sharedMemPerMultiprocessor );
        printf(" ---MP Information for device %d ---\n", i);
        printf("SM count: %d\n", properties.multiProcessorCount);
        printf("Threads in warp: %d\n", properties.warpSize);
        printf("Max threads per SM: %d\n", properties.maxThreadsPerMultiProcessor);
        printf("Max warps per SM: %d\n",properties.maxThreadsPerMultiProcessor/properties.warpSize);
        printf("Max threads per block: %d\n", properties.maxThreadsPerBlock);
        printf("Max thread dimensions: (%d, %d, %d)\n", properties.maxThreadsDim[0], properties.maxThreadsDim[1], properties.maxThreadsDim[2]);
        printf("Max grid dimensions: (%d, %d, %d)\n", properties.maxGridSize[0], properties.maxGridSize[1], properties.maxGridSize[2]); 
    }
    return 0;
    
}

Writing device_properties.cu


In [19]:
%%bash

nvcc --compiler-options -Wall device_properties.cu -o device_properties.out

In [20]:
%%bash

./device_properties.out

----------------------
----device 0 ----
Device Name: Tesla K20Xm
Compute capability: 3.5
Clock rate: 732000
Unified memory: 1
 ---Memory Information for device 0 (results on bytes)---
Total global mem: 5977800704
Total constant Mem: 65536
Shared memory per thread block: 49152
Shared memory per SM: 49152
 ---MP Information for device 0 ---
SM count: 14
Threads in warp: 32
Max threads per SM: 2048
Max warps per SM: 64
Max threads per block: 1024
Max thread dimensions: (1024, 1024, 64)
Max grid dimensions: (2147483647, 65535, 65535)
----------------------
----device 1 ----
Device Name: Tesla K20Xm
Compute capability: 3.5
Clock rate: 732000
Unified memory: 1
 ---Memory Information for device 1 (results on bytes)---
Total global mem: 5977800704
Total constant Mem: 65536
Shared memory per thread block: 49152
Shared memory per SM: 49152
 ---MP Information for device 1 ---
SM count: 14
Threads in warp: 32
Max threads per SM: 2048
Max warps per SM: 64
Max threads per block: 1024
Max thread dim

### Regla compuesta del rectángulo

**$n=10^3$** subintervalos

In [113]:
%%file Rcf.cu
#include<stdio.h>
#include <thrust/reduce.h>
#include <thrust/execution_policy.h>

__global__ void Rcf(double *data, double a, double h_hat, int n, double *sum ) {
    double x=0.0;
    if(threadIdx.x<=n-1){
        x=a+(threadIdx.x+1/2.0)*h_hat;
        data[threadIdx.x]=std::exp(-std::pow(x,2));
    }
    if(threadIdx.x==0){
        *sum = thrust::reduce(thrust::device, data , data + n, (double)0, thrust::plus<double>());
    }
}

int main(int argc, char *argv[]){
    double suma=0.0;
    double *d_data;
    double *d_suma;
    double a=0.0, b=1.0;
    double h_hat;
    int n=1e3; //número de subintervalos
    double objetivo=0.7468241328124271;
    double time_spent;
    clock_t begin,end;
    cudaMalloc((void **)&d_data,sizeof(double)*n);
    cudaMalloc((void**)&d_suma,sizeof(double));
    h_hat=(b-a)/n;
    begin=clock();
    Rcf<<<1,n>>>(d_data, a,h_hat,n,d_suma); //1 bloque de n threads
    cudaDeviceSynchronize();
    end=clock();
    time_spent = (double)(end - begin) / CLOCKS_PER_SEC;
    cudaMemcpy(&suma, d_suma, sizeof(double), cudaMemcpyDeviceToHost);
    suma=h_hat*suma;
    cudaFree(d_data) ;
    cudaFree(d_suma) ;
    printf("Integral de %f a %f = %1.15e\n", a,b,suma);
    printf("Error relativo de la solución: %1.15e\n", fabs(suma-objetivo)/fabs(objetivo));
    printf("Tiempo de cálculo en la gpu %.5f\n", time_spent);
    return 0;
}

Overwriting Rcf.cu


In [106]:
%%bash
nvcc --compiler-options -Wall Rcf.cu -o Rcf.out

In [107]:
%%bash
./Rcf.out

Integral de 0.000000 a 1.000000 = 7.468241634690490e-01
Error relativo de la solución: 4.104931878976858e-08
Tiempo de cálculo en la gpu 0.00035


In [108]:
%%bash
nvprof --normalized-time-unit s ./Rcf.out

Integral de 0.000000 a 1.000000 = 7.468241634690490e-01
Error relativo de la solución: 4.104931878976858e-08
Tiempo de cálculo en la gpu 0.00043


==1343== NVPROF is profiling process 1343, command: ./Rcf.out
==1343== Warning: Profiling results might be incorrect with current version of nvcc compiler used to compile cuda app. Compile with nvcc compiler 9.0 or later version to get correct profiling results. Ignore this warning if code is already compiled with the recommended nvcc version 
==1343== Profiling application: ./Rcf.out
==1343== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
                        %         s                   s         s         s
 GPU activities:    97.33  8.27e-05         1  8.27e-05  8.27e-05  8.27e-05  Rcf(double*, double, double, int, double*)
                     2.67  2.27e-06         1  2.27e-06  2.27e-06  2.27e-06  [CUDA memcpy DtoH]
      API calls:    98.82  0.146833         2  0.073417  8.00e-06  0.146825  cudaMalloc
                     0.37  5.44e-04       194  2.81e-06  1.78e-07  9.54e-05  cuDeviceGetAttribute
                     0.32

$n=1025$ subintervalos

In [186]:
%%file Rcf2.cu
#include<stdio.h>
#include <thrust/reduce.h>
#include <thrust/execution_policy.h>

__global__ void Rcf(double *data, double a, double h_hat, int n, double *sum ) {
    double x=0.0;
    if(threadIdx.x<=n-1){
        x=a+(threadIdx.x+1/2.0)*h_hat;
        data[threadIdx.x]=std::exp(-std::pow(x,2));
    }
    if(threadIdx.x==0){
        *sum = thrust::reduce(thrust::device, data , data + n, (double)0, thrust::plus<double>());
    }
}

int main(int argc, char *argv[]){
    double suma=0.0;
    double *d_data;
    double *d_suma;
    double a=0.0, b=1.0;
    double h_hat;
    int n=1025; //número de subintervalos
    double objetivo=0.7468241328124271;
    double time_spent;
    clock_t begin,end;
    cudaMalloc((void **)&d_data,sizeof(double)*n);
    cudaMalloc((void**)&d_suma,sizeof(double));
    h_hat=(b-a)/n;
    begin=clock();
    Rcf<<<1,n>>>(d_data, a,h_hat,n,d_suma); //1 bloque de n threads
    cudaDeviceSynchronize();
    end=clock();
    time_spent = (double)(end - begin) / CLOCKS_PER_SEC;
    cudaMemcpy(&suma, d_suma, sizeof(double), cudaMemcpyDeviceToHost);
    suma=h_hat*suma;
    cudaFree(d_data) ;
    cudaFree(d_suma) ;
    printf("Integral de %f a %f = %1.15e\n", a,b,suma);
    printf("Error relativo de la solución: %1.15e\n", fabs(suma-objetivo)/fabs(objetivo));
    printf("Tiempo de cálculo en la gpu %.5f\n", time_spent);
    return 0;
}

Overwriting Rcf2.cu


In [187]:
%%bash
nvcc --compiler-options -Wall Rcf2.cu -o Rcf2.out

In [188]:
%%bash
./Rcf2.out

Integral de 0.000000 a 1.000000 = 0.000000000000000e+00
Error relativo de la solución: 1.000000000000000e+00
Tiempo de cálculo en la gpu 0.00001


Obsérvese error relativo de $100\%$

**¿Cómo lo arreglamos?**

In [192]:
%%file Rcf3.cu
#include<stdio.h>
#include <thrust/reduce.h>
#include <thrust/execution_policy.h>

__global__ void Rcf(double *data, double a, double h_hat, int n, double *sum) {
    double x=0.0;
    int stride=0;
    if(threadIdx.x<=n-1){
        x=a+(threadIdx.x+1/2.0)*h_hat;
        data[threadIdx.x]=std::exp(-std::pow(x,2));
    }
    if(threadIdx.x==0){
        stride=blockDim.x;
        x=a+(threadIdx.x+stride+1/2.0)*h_hat;
        data[threadIdx.x+stride]=std::exp(-std::pow(x,2));
        *sum = thrust::reduce(thrust::device, data , data + n, (double)0, thrust::plus<double>());
    }
}

int main(int argc, char *argv[]){
    double suma=0.0;
    double *d_data;
    double *d_suma;
    double a=0.0, b=1.0;
    double h_hat;
    int n_threads_per_block=1024; 
    int n_bloques=2;
    int n=1025;//número de subintervalos
    double objetivo=0.7468241328124271;
    double time_spent;
    clock_t begin,end;
    cudaMalloc((void **)&d_data,sizeof(double)*n);
    cudaMalloc((void**)&d_suma,sizeof(double));
    h_hat=(b-a)/n;
    begin=clock();
    Rcf<<<n_bloques,n_threads_per_block>>>(d_data, a,h_hat,n,d_suma); 
    cudaDeviceSynchronize();
    end=clock();
    time_spent = (double)(end - begin) / CLOCKS_PER_SEC;
    cudaMemcpy(&suma, d_suma, sizeof(double), cudaMemcpyDeviceToHost);
    suma=h_hat*suma;
    cudaFree(d_data) ;
    cudaFree(d_suma) ;
    printf("Integral de %f a %f = %1.15e\n", a,b,suma);
    printf("Error relativo de la solución: %1.15e\n", fabs(suma-objetivo)/fabs(objetivo));
    printf("Tiempo de cálculo en la gpu %.5f\n", time_spent);
    return 0;
}

Overwriting Rcf3.cu


In [193]:
%%bash
nvcc --compiler-options -Wall Rcf3.cu -o Rcf3.out

In [194]:
%%bash
./Rcf3.out

Integral de 0.000000 a 1.000000 = 7.468241619918411e-01
Error relativo de la solución: 3.907133247860604e-08
Tiempo de cálculo en la gpu 0.00034


In [198]:
%%file Rcf4.cu
#include<stdio.h>
#include <thrust/reduce.h>
#include <thrust/execution_policy.h>

__global__ void Rcf(double *data, double a, double h_hat, int n, double *sum) {
    double x=0.0;
    int stride=0;
    int i;
    stride=blockDim.x;
    for(i=threadIdx.x;i<=n-1;i+=stride){
        if(i<=n-1){
            x=a+(i+1/2.0)*h_hat;
            data[i]=std::exp(-std::pow(x,2));
        }
    }
    if(threadIdx.x==0){
        *sum = thrust::reduce(thrust::device, data , data + n, (double)0, thrust::plus<double>());
    }
}

int main(int argc, char *argv[]){
    double suma=0.0;
    double *d_data;
    double *d_suma;
    double a=0.0, b=1.0;
    double h_hat;
    int n_threads_per_block=1024; 
    int n_bloques=2;
    int n=n_threads_per_block*n_bloques;//número de subintervalos
    double objetivo=0.7468241328124271;
    double time_spent;
    clock_t begin,end;
    cudaMalloc((void **)&d_data,sizeof(double)*n);
    cudaMalloc((void**)&d_suma,sizeof(double));
    h_hat=(b-a)/n;
    begin=clock();
    Rcf<<<n_bloques,n_threads_per_block>>>(d_data, a,h_hat,n,d_suma); 
    cudaDeviceSynchronize();
    end=clock();
    time_spent = (double)(end - begin) / CLOCKS_PER_SEC;
    cudaMemcpy(&suma, d_suma, sizeof(double), cudaMemcpyDeviceToHost);
    suma=h_hat*suma;
    cudaFree(d_data) ;
    cudaFree(d_suma) ;
    printf("Integral de %f a %f = %1.15e\n", a,b,suma);
    printf("Error relativo de la solución: %1.15e\n", fabs(suma-objetivo)/fabs(objetivo));
    printf("Tiempo de cálculo en la gpu %.5f\n", time_spent);
    return 0;
}

Overwriting Rcf4.cu


In [199]:
%%bash
nvcc --compiler-options -Wall Rcf4.cu -o Rcf4.out

In [200]:
%%bash
./Rcf4.out

Integral de 0.000000 a 1.000000 = 7.468241401215338e-01
Error relativo de la solución: 9.786918140590463e-09
Tiempo de cálculo en la gpu 0.00046


**¿Más nodos?**

In [201]:
%%file Rcf5.cu
#include<stdio.h>
#include <thrust/reduce.h>
#include <thrust/execution_policy.h>

__global__ void Rcf(double *data, double a, double h_hat, int n, double *sum ) {
    double x=0.0;
    int idx;
    idx = blockIdx.x * blockDim.x + threadIdx.x;
    if(idx<=n-1){
        x=a+(idx+1/2.0)*h_hat;
        data[idx]=std::exp(-std::pow(x,2));
    }
    if(idx==0){
        *sum = thrust::reduce(thrust::device, data , data + n, (double)0, thrust::plus<double>());
    }
}

int main(int argc, char *argv[]){
    double suma=0.0;
    double *d_data;
    double *d_suma;
    double a=0.0, b=1.0;
    double h_hat;
    int n_threads_per_block=1024; 
    int n_bloques=1000;
    double objetivo=0.7468241328124271;
    int n=n_bloques*n_threads_per_block;//número de subintervalos
    double time_spent;
    clock_t begin,end;
    cudaMalloc((void **)&d_data,sizeof(double)*n);
    cudaMalloc((void**)&d_suma,sizeof(double));
    h_hat=(b-a)/n;
    begin = clock();
    Rcf<<<n_bloques,n_threads_per_block>>>(d_data, a,h_hat,n,d_suma); 
    cudaDeviceSynchronize();
    end = clock();
    time_spent = (double)(end - begin) / CLOCKS_PER_SEC;
    cudaMemcpy(&suma, d_suma, sizeof(double), cudaMemcpyDeviceToHost);
    suma=h_hat*suma;
    cudaFree(d_data) ;
    cudaFree(d_suma) ;
    printf("Integral de %f a %f = %1.15e\n", a,b,suma);
    printf("Error relativo de la solución: %1.15e\n", fabs(suma-objetivo)/fabs(objetivo));
    printf("Tiempo de cálculo en la gpu %.5f\n", time_spent);
    return 0;
}

Writing Rcf5.cu


In [202]:
%%bash
nvcc --compiler-options -Wall Rcf5.cu -o Rcf5.out

In [203]:
%%bash
./Rcf5.out

Integral de 0.000000 a 1.000000 = 7.468241328124601e-01
Error relativo de la solución: 4.415179207880368e-14
Tiempo de cálculo en la gpu 0.11359


In [204]:
%%bash
nvprof --normalized-time-unit s ./Rcf5.out

Integral de 0.000000 a 1.000000 = 7.468241328124601e-01
Error relativo de la solución: 4.415179207880368e-14
Tiempo de cálculo en la gpu 0.11382


==2537== NVPROF is profiling process 2537, command: ./Rcf5.out
==2537== Warning: Profiling results might be incorrect with current version of nvcc compiler used to compile cuda app. Compile with nvcc compiler 9.0 or later version to get correct profiling results. Ignore this warning if code is already compiled with the recommended nvcc version 
==2537== Profiling application: ./Rcf5.out
==2537== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
                        %         s                   s         s         s
 GPU activities:   100.00  0.113309         1  0.113309  0.113309  0.113309  Rcf(double*, double, double, int, double*)
                     0.00  2.37e-06         1  2.37e-06  2.37e-06  2.37e-06  [CUDA memcpy DtoH]
      API calls:    56.61  0.151360         2  0.075680  1.41e-04  0.151218  cudaMalloc
                    42.38  0.113325         1  0.113325  0.113325  0.113325  cudaDeviceSynchronize
                     0

**Referencias**

1. N. Matloff, Parallel Computing for Data Science. With Examples in R, C++ and CUDA, 2014.

2. [CUDA](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/tree/master/C/extensiones_a_C/CUDA)

3. [2.3.CUDA](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/II.computo_paralelo/2.3.CUDA.ipynb)

Para más sobre *Unified Memory* revisar:

* [Even easier introduction to cuda](https://devblogs.nvidia.com/even-easier-introduction-cuda/)

* [Unified memory cuda beginners](https://devblogs.nvidia.com/unified-memory-cuda-beginners/)